# Maestria Big Data & Bussines Analitics

## Alumno: Cristian Ortiz

### Social Network Analisis

#### Extraer 2000 tweets de un tema (hashtag) de Twitter, generar un CSV con la informacion deseada, estructurar la informacion de tal forma de crear un Grafo para despues procesarlo con Gephi


## Procedimiento

### Se importan las librerias necesarias

In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
# To add wait time between requests
import time
import networkx as nx

#### Se declara una variable de entorno para el manejo del Bearer Token, este procedimiento tambien puede hacerlo poniendo las credenciales de la API de Twitter dentro de un archivo CFG y leerlo, pero por facilidad abajo se ha declarado y colocado la variable con la credencial necesaria

In [2]:
# Enviroment variable
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAFNlWQEAAAAA8Y8DQIQ9gIeEa24R6sMqEz9ejUs%3D2BGfEy35au0A0lnDwi7WduGERatp127rEaqfWKlUxW2jwuQC0W'

#### Las siguientes funciones arman el encabezado con la credencial y en lo posterior poder realizar la consulta

In [3]:
# For get enviroment variable
def auth():
    return os.getenv('TOKEN')

In [4]:
# Authorization through Bearer Token
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

#### Para la peticion se construye una funcion para manejar diferentes variables dentro de la URL, por defecto el maximo de resultados por consulta es de 100.

#### Adicionalmente se solicita la variable: referenced_tweets.id.author_id, debido a que se extraeran tambien retweets, replies, quotes y se necesita saber el author original del retweet para asi establecer edges(retweets, in replies, quotes) entre nodos 

In [5]:
# Build a url with all request parameters
# Take care about recent/all endpoints
def create_url(keyword,start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'entities.mentions.username,referenced_tweets.id.author_id,author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

#### La siguiente funcion se conecta a la url establecida (search/recent) definiendo un comando GET, url, headers viene a ser la funcion de armado de la llave con el Bearer y params para el manejo del next token, ademas tiene un display de errores (consultar manual de errores)
#### Al pricipio este codigo trabajaba con la url: "https://api.twitter.com/2/tweets/search/all" pero la cueta de desarrollador estandar solo permite consultar tweets recientes. Para consultar todos los tweets se necesita la cuenta ACADEMIC RESEARCH

In [6]:
# For make requests with seach recent
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

#### Para poder definir adecuadamente esta funcion se tuvo que observar a detalle la respuesta en formato JSON que ofrece la API, e iterar adecuadamente sobre los diferentes diccionarios para guardar los valores de las variables con las que se ira trabajando.

In [7]:
# For attatch every request to CSV
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 1. Author ID
        author_id = tweet['author_id']

        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 3. Geolocation
        #if ('geo' in tweet):   
        #    geo = tweet['geo']['place_id']
        #else:
         #   geo = " "

        # 4. Tweet ID
        tweet_id = tweet['id']

        # 5. Language
        lang = tweet['lang']

        # 6. Tweet metrics
        retweet_count = tweet['public_metrics']['retweet_count']
        reply_count = tweet['public_metrics']['reply_count']
        like_count = tweet['public_metrics']['like_count']
        quote_count = tweet['public_metrics']['quote_count']

        # 7. source
        source = tweet['source']

        # 8. Tweet text
        text = tweet['text']
        
        # 9. Retweet Username
       
        if ('referenced_tweets' in tweet):# en el caso que sea un retweet
            if tweet['referenced_tweets'][0]['type'] == 'retweeted':
                retweeted_by = tweet['referenced_tweets'][0]['id']
                replied_to = " "
                quoted_by = " "
                author_retweet = " "
                reply_author = " "
                quote_author = " "
                for tweet2 in json_response['includes']['tweets']:
                    if retweeted_by == tweet2['id']:
                        author_retweet= tweet2['author_id']
                        break
            if tweet['referenced_tweets'][0]['type'] == 'replied_to':# en el caso que sea un replied to
                replied_to = tweet['referenced_tweets'][0]['id']
                retweeted_by = " "
                quoted_by = " "
                author_retweet = " "
                reply_author = " "
                quote_author = " "
                for tweet2 in json_response['includes']['tweets']:
                    if replied_to == tweet2['id']:
                        reply_author = tweet2['author_id']
                        break
            if tweet['referenced_tweets'][0]['type'] == 'quoted':# en el caso que sea un quote
                quoted_by = tweet['referenced_tweets'][0]['id']
                retweeted_by = " "
                replied_to = " "
                author_retweet = " "
                reply_author = " "
                quote_author = " "
                for tweet2 in json_response['includes']['tweets']:
                    if quoted_by == tweet2['id']:
                        quote_author = tweet2['author_id']
                        break
        #    en el caso que sea un tweet original
        else:
            retweeted_by = " "
            replied_to = " "
            quoted_by = " "
            author_retweet = " "
            reply_author = " "
            quote_author = " "
        #if ('referenced_tweets' in json_response['includes']['tweets']): 
            
            
            
        #for tweet2 in json_response['includes']['tweets']:
         #   if tweet2['id'] == (retweeted_by or replied_to or quoted_by):
          #      author_referenced = tweet2['author_id']
            #else:
               # author_referenced = " "
                
        #for tweet in json_response['includes']['users']:
        #user_id = tweet['id']
        
        # Considerar el orden de las variables dentro del arreglo que coincida con el encabezado del csv
        res = [author_id, author_retweet, reply_author, quote_author, source, tweet_id, like_count, retweet_count, quote_count, reply_count, text]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1
        
    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

#### Se define el nombre del archivo y su encabezado, cuyo orden debe coincidir con el establecido en la variable res

In [8]:
csvFile = open("blockchain_results.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)
#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author_id','author_retweet','reply_author','quote_author','source','tweet_id','like_count','retweet_count','quote_count','reply_count','text'])
csvFile.close()

#### Los siguientes valores se pueden modificar a voluntad, considerando times dentro de los ultimos 7 dias, caso contrario saldra un error.
#### Al principio se estuvo trabajando sin fechas, obteniendo asi, resultados que, por ser relativamente nuevos, no tenian retweets, likes, etc., dificultando asi la implementacion de edges entre nodos. Tambien se obtuvieron resultados de usuarios verificados con dificultades similares.

In [9]:
keyword = "#blockchain lang:en"
start_time = ['2021-12-14T00:00:00.000Z']
end_time = ['2021-12-17T00:00:00.000Z']
max_count = 2000 # Ingresar la cantidad maxima de tweets requeridos

#### Se establecen contadores, banderas y demas valores que no se recomienda modificar

In [10]:
total_tweets = 0
count = 0 # Counting tweets per time period
flag = True
next_token = None
max_results = 100

#### Se arma la url

In [11]:
url = create_url(keyword, start_time, end_time, max_results)

#### El siguiente codigo fue tomado de las fuentes citadas al final, se encarga de guardar los resultados de cada pagina de la consulta. Al ejecutarlo tomara un tiempo considerable y segun el numero de resultados requerido

In [12]:
bearer_token = auth()
headers = create_headers(bearer_token)
while flag:
       # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_time, end_time, max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                #print("Start Date: ", start_list[i])
                append_to_csv(json_response, "blockchain_results.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(2)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                #print("Start Date: ", start_list[i])
                append_to_csv(json_response, "blockchain_results.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(2)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(2)
print("Total number of results: ", total_tweets)

-------------------
Token:  None
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe15rkdxx2s27itux62cx72oob2bh
# of Tweets added from this response:  100
Total # of Tweets added:  100
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe15rkdxx2s27itux62cx72oob2bh
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe15rkdxx1psyzub5jjbzm84njw59
# of Tweets added from this response:  99
Total # of Tweets added:  199
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe15rkdxx1psyzub5jjbzm84njw59
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe15rkdnib09xe14rltthox8ydjel
# of Tweets added from this response:  100
Total # of Tweets added:  299
-------------------
-------------------
Token:  b26v89c19zqg8o3fpe15rkdnib09xe14rltthox8ydjel
Endpoint Response Code: 200
Next Token:  b26v89c19zqg8o3fpe15rkdni9xwd0j022kaoyqo5dfnh
# of Tweets added from this response:  100
Total # of Tweets added:  399
-------------------
-------------------
Token:

#### Se puede observar el Json completo o por sus diccionarios con el codigo de abajo y descomentando lo requerido

In [13]:
#json_response#['includes']['tweets']#['data']#['includes']['tweets']#['meta']

#### Al no poder generar un grafo con Gephi con el archivo CSV obtenido, se tiene que crear el grafo con la libreria NetworkX, se declara una variable G que viene a ser el grafo.

In [14]:
G = nx.DiGraph()
#len(G.nodes)

#### Se crea un dataframe a partir del csv generado

In [15]:
data = pd.read_csv('blockchain_results.csv')#,header=None)
df = pd.DataFrame(data)
#df.iloc[:,2]
df.head(15)#['user_id']
#row[0]

,author_id,author_retweet,reply_author,quote_author,source,tweet_id,like_count,retweet_count,quote_count,reply_count,text
0,1382897145150373889,,,,Twitter,1471631223785234434,0,0,0,0,Current stats of DELEGATE_DONT_HATE\nBlocks Fo...
1,1296773574427115521,,,,Twitter,1471631213949595648,0,0,0,1,🐳 #Cardano $ADA Whale ❤️laced!\n💰 Transaction ...
2,1449888673000943616,,,,Twitter Web App,1471631209855799297,1,0,0,1,Moon Soon 📈💸🚀\n\n#blockchaingaming #Blockchain...
3,1296773574427115521,,,,Twitter,1471631209822359555,0,0,0,1,🐳 #Cardano $ADA Whale ❤️laced!\n💰 Transaction ...
4,1383664865550561285,1433638298887491601,,,Twitter Web App,1471631205934125058,0,3732,0,0,RT @Digi_metaverse: Here's a sneak peek of the...
5,1393005306377084934,1397944006982750214,,,Twitter for Android,1471631183058542596,0,6,0,0,RT @Nafty_Official: Start making more money fr...
6,1412562456312913921,1459775026547331072,,,Twitter Web App,1471631178822078466,0,2862,0,0,RT @bitfrenchies: The #BFS team has been worki...
7,1460122250946834432,242782539,,,Twitter for Android,1471631172249985024,0,1397,0,0,RT @bertawagnerger: 🧑‍🚀Anons Network\nThe meme...
8,1469852546533449729,1412611156301156360,,,Twitter for Android,1471631168714031104,0,3,0,0,RT @Penguin_DSN: I just published #Pen #DSN Us...
9,290641102,,,,Periscope,1471631158655926277,3,3,0,1,#FiredUp Live #Toronto #HappyHolidays #MerryCh...


#### Los nodos vienen a ser todos los usuarios de tweets, retweets, quotes and replies

In [16]:
G.add_nodes_from(df['author_id'].tolist())
print("Total de nodos:",len(G.nodes))

Total de nodos: 1722


#### Para los pesos de los nodos, se ha sumado los likes, repliest, retweets y quotes que tiene el tweet de dicho usuario. Se debe considerar el multiplicar los retweets por un coeficiente de tal forma que tenga menor influencia en el peso del nodo.

In [17]:
total1 = df['like_count'] + df['quote_count'] + df['reply_count'] + (df['retweet_count'] )
nx.set_node_attributes(G, dict(zip(df['author_id'].tolist(), total1.tolist())),'weight')
#G.nodes

#### Para definir aristas se va a establecer relacion entre nodos en el caso de haber referencias al tweet, con sus respectivos pesos

In [18]:
#aristas
for index, row in df.iterrows():
    node = row['author_id']
    if row['author_retweet'] != " ":
        G.add_edge(node, row[1])
    if row['reply_author']!= " ":
        G.add_edge(node, row[2])
    if row['quote_author']!= " ":
        G.add_edge(node, row[3])   

#### Por ultimo se exporta el grafo

In [19]:
nx.write_graphml(G, "blockchain_grafo.graphml" )
G.nodes

NodeView((1382897145150373889, 1296773574427115521, 1449888673000943616, 1383664865550561285, 1393005306377084934, 1412562456312913921, 1460122250946834432, 1469852546533449729, 290641102, 1447734848437714948, 1451590302880333826, 1442431789041999874, 1471072856117178368, 1454082648851116043, 1389272391872925701, 1398921775233175552, 1319598690328178690, 1470312094872772612, 708549109360300032, 1470988459619676162, 1429369738942160902, 1401682282910568448, 1412352718719442946, 1455336035433418752, 1406736283800506371, 1373815960008863746, 1381486659221458946, 997855218011463680, 905827759234191361, 1407363183518437380, 1386050203363397633, 1387600040709935106, 1468227922515349513, 2533092433, 1394668557867253765, 2365487068, 56851432, 1446755916913598468, 1471058136668655617, 1441293880234692611, 1471440152740646912, 1374481929701969926, 1216660997928173568, 1432604316062285824, 1413806440230260736, 1321244181105463298, 1462010201419051010, 950800296653008896, 1286079547809521664, 1432

### Fuentes relevantes: 
-https://towardsdatascience.com/an-extensive-guide-to-collecting-tweets-from-twitter-api-v2-for-academic-research-using-python-3-518fcb71df2a

-https://twittercommunity.com/t/twitter-api-v2-getting-the-full-text-of-the-referenced-tweet-when-doing-a-recent-search/153083/10

-https://networkx.org/documentation/stable/tutorial.html